In [1]:
import os
import xmlrpc.client
import pandas as pd
from pathlib import Path

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

In [2]:
search_fact = ["&", "&", "&",
          ("state", "=", "posted"),
          ("invoice_date", ">=", "2024-01-01"), 
          ("invoice_date", "<=", "2024-03-31"), 
          ("journal_id", "in", [10, 90, 30, 97])]

fact_doc_fields = [
          'name',
          'invoice_date',
          'state',
          'reversed_entry_id',
          'reversal_move_id',
          'move_type',
          ]

fact_doc_new_ids = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [search_fact])
fact_doc_new_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_new_ids], {'fields': fact_doc_fields})
fact_doc_new_df = pd.DataFrame(fact_doc_new_json)

fact_doc_new_df.loc[:, 'reversed_entry_id'] = fact_doc_new_df.loc[:, 'reversed_entry_id'].str.get(0).astype('Int64')

fact_doc_new_df

,id,name,invoice_date,state,reversed_entry_id,reversal_move_id,move_type
0,46062,RF1-CC/2024/00034,2024-03-21,posted,45890,[],out_refund
1,46344,F2-VS/2024/00631,2024-03-21,posted,<NA>,[],out_invoice
2,46338,F2-VS/2024/00630,2024-03-21,posted,<NA>,[],out_invoice
3,46223,F2-VS/2024/00629,2024-03-21,posted,<NA>,[],out_invoice
4,46205,F2-VS/2024/00628,2024-03-21,posted,<NA>,[],out_invoice
...,...,...,...,...,...,...,...
11096,149,F1-CC/2024/00008,2024-01-02,posted,<NA>,[],out_invoice
11097,125,F1-CC/2024/00004,2024-01-02,posted,<NA>,[],out_invoice
11098,122,F1-CC/2024/00003,2024-01-02,posted,<NA>,[],out_invoice
11099,117,F1-CC/2024/00002,2024-01-02,posted,<NA>,[],out_invoice


In [3]:
fact_doc_new_df.loc[fact_doc_new_df['move_type'] == 'out_refund']

,id,name,invoice_date,state,reversed_entry_id,reversal_move_id,move_type
0,46062,RF1-CC/2024/00034,2024-03-21,posted,45890,[],out_refund
102,45801,RF2-CC/2024/00049,2024-03-20,posted,45754,[],out_refund
299,45201,RF2-CC/2024/00048,2024-03-19,posted,45121,[],out_refund
300,44928,RF2-CC/2024/00047,2024-03-19,posted,44925,[],out_refund
301,44742,RF2-CC/2024/00046,2024-03-19,posted,44288,[],out_refund
...,...,...,...,...,...,...,...
10272,3192,RF1-CC/2024/00003,2024-01-08,posted,3174,[],out_refund
10453,2492,RF1-CC/2024/00002,2024-01-06,posted,2456,[],out_refund
10621,1349,RF2-CC/2024/00002,2024-01-05,posted,187,[],out_refund
10938,882,RF2-CC/2024/00001,2024-01-03,posted,879,[],out_refund


In [4]:
fact_doc_new_df.loc[(fact_doc_new_df['move_type'] == 'out_refund') & (fact_doc_new_df['reversed_entry_id'].isna())]

,id,name,invoice_date,state,reversed_entry_id,reversal_move_id,move_type
302,45175,RF1-CC/2024/00033,2024-03-19,posted,<NA>,[],out_refund
2002,37628,RF1-VS/2024/00004,2024-03-06,posted,<NA>,[],out_refund
4622,26436,RF1-VS/2024/00003,2024-02-16,posted,<NA>,[],out_refund
5738,21450,RF1-VS/2024/00002,2024-02-08,posted,<NA>,[],out_refund


In [5]:
NCdf = fact_doc_new_df.loc[(fact_doc_new_df['move_type'] == 'out_refund') & (~fact_doc_new_df['reversed_entry_id'].isna())]
NCdf

,id,name,invoice_date,state,reversed_entry_id,reversal_move_id,move_type
0,46062,RF1-CC/2024/00034,2024-03-21,posted,45890,[],out_refund
102,45801,RF2-CC/2024/00049,2024-03-20,posted,45754,[],out_refund
299,45201,RF2-CC/2024/00048,2024-03-19,posted,45121,[],out_refund
300,44928,RF2-CC/2024/00047,2024-03-19,posted,44925,[],out_refund
301,44742,RF2-CC/2024/00046,2024-03-19,posted,44288,[],out_refund
...,...,...,...,...,...,...,...
10272,3192,RF1-CC/2024/00003,2024-01-08,posted,3174,[],out_refund
10453,2492,RF1-CC/2024/00002,2024-01-06,posted,2456,[],out_refund
10621,1349,RF2-CC/2024/00002,2024-01-05,posted,187,[],out_refund
10938,882,RF2-CC/2024/00001,2024-01-03,posted,879,[],out_refund


In [6]:
reversed_entry_ids = NCdf['reversed_entry_id']

rev_ids_list = []
for rid in reversed_entry_ids:
    rev_ids_list.append(int(rid))

rev_ids_list

fact_doc_from_reversed_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [rev_ids_list], {'fields': fact_doc_fields})
fact_doc_from_reversed_df = pd.DataFrame(fact_doc_from_reversed_json)
fact_doc_from_reversed_df.loc[:, 'reversal_move_id'] = fact_doc_from_reversed_df.loc[:, 'reversal_move_id'].str.get(0).astype('Int64')

fact_doc_from_reversed_df

,id,name,invoice_date,state,reversed_entry_id,reversal_move_id,move_type
0,45890,F1-CC/2024/04168,2024-03-20,posted,False,46062,out_invoice
1,45754,F2-CC/2024/05568,2024-03-20,posted,False,45801,out_invoice
2,45121,F2-CC/2024/05497,2024-03-19,cancel,False,45201,out_invoice
3,44925,F2-CC/2024/05470,2024-03-19,cancel,False,44928,out_invoice
4,44288,F2-CC/2024/05392,2024-03-18,posted,False,44742,out_invoice
...,...,...,...,...,...,...,...
66,3174,F1-CC/2024/00314,2024-01-08,posted,False,3192,out_invoice
67,2456,F1-CC/2024/00253,2024-01-06,posted,False,2492,out_invoice
68,187,F2-CC/2024/00018,2024-01-02,posted,False,1349,out_invoice
69,879,F2-CC/2024/00103,2024-01-03,posted,False,882,out_invoice


In [7]:
dffull = NCdf.merge(fact_doc_from_reversed_df.loc[fact_doc_from_reversed_df['state'] == 'cancel'], how='right', left_on='id', right_on='reversal_move_id')
df = dffull[['id_x', 'name_x', 'invoice_date_x', 'state_x', 'id_y', 'name_y', 'invoice_date_y',
       'state_y']]
df

,id_x,name_x,invoice_date_x,state_x,id_y,name_y,invoice_date_y,state_y
0,45201,RF2-CC/2024/00048,2024-03-19,posted,45121,F2-CC/2024/05497,2024-03-19,cancel
1,44928,RF2-CC/2024/00047,2024-03-19,posted,44925,F2-CC/2024/05470,2024-03-19,cancel
2,39736,RF2-CC/2024/00040,2024-03-09,posted,39505,F2-CC/2024/04878,2024-03-09,cancel
3,39716,RF2-CC/2024/00039,2024-03-09,posted,39459,F2-CC/2024/04869,2024-03-09,cancel
4,31290,RF2-CC/2024/00036,2024-02-26,posted,31240,F2-CC/2024/03970,2024-02-26,cancel
5,29473,RF2-CC/2024/00035,2024-02-22,posted,29236,F2-CC/2024/03708,2024-02-22,cancel
6,28937,RF2-CC/2024/00034,2024-02-21,posted,28811,F2-CC/2024/03665,2024-02-21,cancel
7,28649,RF2-CC/2024/00033,2024-02-21,posted,28400,F2-CC/2024/03611,2024-02-21,cancel
8,24937,RF2-CC/2024/00027,2024-02-14,posted,24899,F2-CC/2024/03188,2024-02-14,cancel
9,24926,RF2-CC/2024/00026,2024-02-14,posted,24001,F2-CC/2024/03052,2024-02-12,cancel


In [8]:
archivo = "Notas de Crédito Vigentes vs Facturas Canceladas"

path = Path.home().joinpath('Desktop').joinpath(archivo +  ".xlsx")

df.to_excel(path)